In [12]:
import pandas as pd                        
from pytrends.request import TrendReq

In [25]:
topics = {
    "Angel City FC": "/g/11jk0bhp11",
    "Chicago Red Stars": "/m/03m6d4g",
    "Houston Dash": "/m/0zmx026",
    "Kansas City Current": "/g/11qplpp6kc",
    "North Carolina Courage": "/g/11c2qz9bgz",
    "NJ/NY Gotham FC": "/m/04lj4rq",
    "OL Reign": "/m/0pb4c5m",
    "Orlando Pride": "/g/11bw8dx_7t",
    "Portland Thorns": "/m/0p77cc5",
    "Racing Louisville": "/g/11jw6dbpr5",
    "San Diego Wave FC": "/g/11q2k8gyhm",
    "Washington Spirit": "/m/0p784vx",
}
from_topics = {v: k for k, v in topics.items()}

In [29]:
def get_data(year, terms):
    pytrend = TrendReq(retries=3, backoff_factor=0.4)

    # Because you can only request 5 teams at a time, we need to break the list into
    # chunks and find which team has the highest trend in the chunk.
    n = 5
    top_interest = []
    for i in range(0, len(terms), n):
        pytrend.build_payload(
            kw_list=list(map(lambda x: topics[x], terms[i : i + n])),
            geo="US",
            timeframe=f"{str(year)}-01-01 {str(year)}-12-31",
        )
        df = pytrend.interest_over_time()
        top_interest.append(df.max().sort_values(ascending=False).index[0])

    # Now we take those top teams to find which had the highest search interest.
    pytrend.build_payload(
        kw_list=top_interest,
        geo="US",
        timeframe=f"{str(year)}-01-01 {str(year)}-12-31",
    )
    df = pytrend.interest_over_time()
    top = df.max().sort_values(ascending=False).index[0]

    # Finally, we request data for chunks of 4 teams with the top team added to
    # normalize.
    results = pd.DataFrame()
    for i in range(0, len(terms), n - 1):
        pytrend.build_payload(
            kw_list=list(
                set(list(map(lambda x: topics[x], terms[i : i + n - 1])) + [top])
            ),
            geo="US",
            timeframe=f"{str(year)}-01-01 {str(year)}-12-31",
        )
        df = pytrend.interest_over_time()
        results = pd.concat([results, df], axis=1)
        
    # Map column names to team names
    results.columns = list(map(lambda x: from_topics[x] if x in from_topics else x , results.columns))

    # Remove duplicate columns
    results = results.loc[:, ~results.columns.duplicated()].copy()

    # Return the ordered results
    return results[terms]

In [30]:
nwsl_teams_2022 = [
    "Angel City FC",
    "Chicago Red Stars",
    "Houston Dash",
    "Kansas City Current",
    "North Carolina Courage",
    "NJ/NY Gotham FC",
    "OL Reign",
    "Orlando Pride",
    "Portland Thorns",
    "Racing Louisville",
    "San Diego Wave FC",
    "Washington Spirit",
]

In [33]:
# Get the 2022 data for all NWSL teams
data = get_data(2022, nwsl_teams_2022)
# Transpose for easier plotting
df = data.transpose()
# Write to csv
df.to_csv("nwsl_2022_trends.csv")

data

,Angel City FC,Chicago Red Stars,Houston Dash,Kansas City Current,North Carolina Courage,NJ/NY Gotham FC,OL Reign,Orlando Pride,Portland Thorns,Racing Louisville,San Diego Wave FC,Washington Spirit
date,,,,,,,,,,,,
2022-01-02,6,2,1,4,3,0,1,2,5,3,0,3
2022-01-09,7,3,5,4,3,0,2,3,6,0,0,4
2022-01-16,4,4,1,1,2,3,4,4,6,0,1,5
2022-01-23,5,1,5,1,1,0,6,4,8,4,0,3
2022-01-30,14,4,6,0,2,4,5,3,7,2,0,7
2022-02-06,13,6,0,0,3,6,7,5,6,2,0,10
2022-02-13,16,5,5,0,1,3,10,5,8,2,0,3
2022-02-20,11,5,4,0,4,5,4,4,8,2,0,6
2022-02-27,14,8,5,1,1,4,8,4,12,3,1,6


In [32]:
# Get the 2021 data for all NWSL teams
data = get_data(2021, nwsl_teams_2022)
# Transpose for easier plotting
df = data.transpose()
# Write to csv
df.to_csv("nwsl_2021_trends.csv")

data

,Angel City FC,Chicago Red Stars,Houston Dash,Kansas City Current,North Carolina Courage,NJ/NY Gotham FC,OL Reign,Orlando Pride,Portland Thorns,Racing Louisville,San Diego Wave FC,Washington Spirit
date,,,,,,,,,,,,
2021-01-03,5,4,6,2,1,4,1,3,5,2,0,0
2021-01-10,9,6,4,9,3,6,2,4,7,7,0,3
2021-01-17,5,4,8,4,2,0,5,2,10,3,0,0
2021-01-24,6,11,1,3,7,0,4,2,7,2,0,2
2021-01-31,5,6,2,2,3,0,5,5,4,1,0,3
2021-02-07,3,7,2,6,1,3,1,2,8,0,0,1
2021-02-14,11,18,1,4,4,3,6,9,8,3,0,9
2021-02-21,7,13,2,5,4,3,4,11,16,3,0,4
2021-02-28,7,10,5,6,7,3,5,13,3,0,0,3
